In [2]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector

# Function to connect to MySQL database
def connect_to_db():
    try:
        # Update with your MySQL database credentials
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Godscout@41",
            database="toanddo_list"
        )
        return conn
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error: {err}")
        return None

# Function to create tasks table if not exists
def create_tasks_table(conn):
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS tasks (
            task_id INT AUTO_INCREMENT PRIMARY KEY,
            task_name VARCHAR(255) NOT NULL,
            task_description TEXT,
            due_date DATE,
            completed BOOLEAN NOT NULL DEFAULT 0
        )
    """)
    conn.commit()

# Function to insert a task into the database
def add_task():
    task_name = task_name_entry.get()
    task_description = task_description_entry.get("1.0", tk.END).strip()
    due_date = due_date_entry.get()

    if task_name:
        conn = connect_to_db()
        if conn:
            cursor = conn.cursor()
            insert_query = """
                INSERT INTO tasks (task_name, task_description, due_date)
                VALUES (%s, %s, %s)
            """
            task_data = (task_name, task_description, due_date)
            cursor.execute(insert_query, task_data)
            conn.commit()
            messagebox.showinfo("Success", "Task added successfully.")
            load_tasks_from_db()
            clear_entries()
            conn.close()

# Function to load tasks from database into the listbox
def load_tasks_from_db():
    conn = connect_to_db()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT task_id, task_name FROM tasks WHERE completed = 0")
        tasks = cursor.fetchall()
        listbox_tasks.delete(0, tk.END)
        for task in tasks:
            listbox_tasks.insert(tk.END, (task[0], task[1]))
        conn.close()

# Function to mark a task as completed in the database
def mark_completed():
    try:
        task_index = listbox_tasks.curselection()[0]
        task_id = listbox_tasks.get(task_index)[0]

        conn = connect_to_db()
        if conn:
            cursor = conn.cursor()
            update_query = """
                UPDATE tasks
                SET completed = 1
                WHERE task_id = %s
            """
            cursor.execute(update_query, (task_id,))
            conn.commit()
            conn.close()
            load_tasks_from_db()
    except IndexError:
        messagebox.showwarning("Warning", "Please select a task to mark as completed.")

# GUI Setup
root = tk.Tk()
root.title("To-Do List with MySQL")

# Create database connection and tasks table
conn = connect_to_db()
if conn:
    create_tasks_table(conn)
    conn.close()

# Task Name Label and Entry
task_name_label = tk.Label(root, text="Task Name:")
task_name_label.pack(pady=5)
task_name_entry = tk.Entry(root, width=50)
task_name_entry.pack()

# Task Description Label and Entry
task_description_label = tk.Label(root, text="Task Description:")
task_description_label.pack(pady=5)
task_description_entry = tk.Text(root, width=50, height=4)
task_description_entry.pack()

# Due Date Label and Entry
due_date_label = tk.Label(root, text="Due Date (YYYY-MM-DD):")
due_date_label.pack(pady=5)
due_date_entry = tk.Entry(root, width=50)
due_date_entry.pack()

# Add Task Button
add_button = tk.Button(root, text="Add Task", width=20, command=add_task)
add_button.pack(pady=10)

# Task Listbox
listbox_tasks = tk.Listbox(root, width=60)
listbox_tasks.pack(pady=10)

# Mark Completed Button
mark_completed_button = tk.Button(root, text="Mark Completed", width=20, command=mark_completed)
mark_completed_button.pack()

# Function to clear all input entries
def clear_entries():
    task_name_entry.delete(0, tk.END)
    task_description_entry.delete("1.0", tk.END)
    due_date_entry.delete(0, tk.END)

# Load tasks initially
load_tasks_from_db()

# Start the main loop
root.mainloop()
